# Geo part

In [1]:
#!conda install -c conda-forge geopy --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         393 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5

In [51]:
from geopy.distance import distance
from geopy.distance import great_circle

import json, logging
from geopy.geocoders import Nominatim

def seperator(a,b):
    c = (b - a)/9
    return [round(a,7),round(a+c,7),round(a+c*2,7),round(a+c*3,7),round(a+c*4,7),round(a+c*5,7),round(a+c*6,7),round(a+c*7,7),round(a+c*8,7),round(b,7)]

def geo_point_creator(city):
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(city)
    lat_list = seperator(float(location.raw['boundingbox'][0]),float(location.raw['boundingbox'][1]))
    lon_list = seperator(float(location.raw['boundingbox'][2]),float(location.raw['boundingbox'][3]))
    point_list = []
    for lat in lat_list:
        for lon in lon_list:
            point_list.append([city,lat,lon])
    df_city = pd.DataFrame(point_list, columns=['city','lat','lng'])
    point0 = [float(df_city.loc[0,['lat']]),float(df_city.loc[0,['lng']])]
    point1 = [float(df_city.loc[1,['lat']]),float(df_city.loc[1,['lng']])]
    point10 = [float(df_city.loc[10,['lat']]),float(df_city.loc[10,['lng']])]
    rad1 = int(distance(point0, point1).meters//2)
    rad2 = int(distance(point0, point10).meters//2) 
    radius = (rad1 + rad2)//2
    #print(point0,point1,point10,rad1,rad2,radius)
    df_city['radius'] = radius
    return df_city

# Data Cleaning

In [2]:
import pandas as pd
import numpy as np
import requests
import re

df = pd.read_csv('CITIES_12062020101058739.csv')

In [34]:
gdp_2018 = df[df['Year']==2018]
top5_gdpt_city = gdp_2018[gdp_2018['VAR']=='GDP_REAL_PPP'].sort_values(by='Value',ascending=False).reset_index().head()
for i,n in enumerate(top5_gdpt_city['METRO_ID'].to_list()):
    x = ''.join(re.findall(r'[A-Za-z]', n))
    top5_gdpt_city['Flags'][i] = top5_gdpt_city['Metropolitan areas'].to_list()[i].replace(' (Greater)','') + ', ' + x

gdpt = top5_gdpt_city['Flags'].to_list()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [35]:
top5_gdpp_city = gdp_2018[gdp_2018['VAR']=='GDP_PC_REAL_PPP'].sort_values(by='Value',ascending=False).reset_index().head()
for i,n in enumerate(top5_gdpp_city['METRO_ID'].to_list()):
    x = ''.join(re.findall(r'[A-Za-z]', n))
    top5_gdpp_city['Flags'][i] = top5_gdpp_city['Metropolitan areas'].to_list()[i].replace(' (Greater)','') + ', ' + x
gdpp = top5_gdpp_city['Flags'].to_list()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [6]:
CLIENT_ID = 'GBHOGHOE3DL1WYFOB4W5WI5SM1PLH1MDY4WN3O12SFGOVDUU' # your Foursquare ID
CLIENT_SECRET = 'AI25GODMBYEC331GD1VSY4Y3MRBP3SKR15N2YC01VKLSR0U4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT = 100

Your credentails:
CLIENT_ID: GBHOGHOE3DL1WYFOB4W5WI5SM1PLH1MDY4WN3O12SFGOVDUU
CLIENT_SECRET:AI25GODMBYEC331GD1VSY4Y3MRBP3SKR15N2YC01VKLSR0U4


In [5]:
def getNearbyVenues(names, latitudes, longitudes, radiuses):
    
    venues_list=[]
    for name, lat, lng, radius in zip(names, latitudes, longitudes, radiuses):
        print('{},{},{}'.format(lat,lng,radius))
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City Name',
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

**for loop to generate the DataFrame**

In [53]:

city_list = ['New York, USA',
 'Los Angeles, USA',
 'Seoul, KOR',
 'Washington, USA',
 'Paris, FR']

In [ ]:
city_point = pd.DataFrame(columns=['city','lat','lng','radius'])
for city in city_list:
    df_tmp = geo_point_creator(city)
    city_point = city_point.append(df_tmp, ignore_index=True)

# Get Venues and Create features

# Test Zone

## Task: Change the city with country code to have better geopy result (Done)

In [63]:
list1 = [['USA145','New York',''],['UK18','London','']]
df = pd.DataFrame(list1,columns=['Country','city','city_country'])
df

,Country,city,city_country
0,USA145,New York,
1,UK18,London,


In [25]:

for i,n in enumerate(top5_gdpt_city['METRO_ID'].to_list()):
    x = ''.join(re.findall(r'[A-Za-z]', n))
    top5_gdpt_city['Flags'][i] = top5_gdpt_city['Metropolitan areas'].to_list()[i].replace(' (Greater)','') + ', ' + x

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [14]:
gdp_2018 = df[df['Year']==2018]
top5_gdpt_city = gdp_2018[gdp_2018['VAR']=='GDP_REAL_PPP'].sort_values(by='Value',ascending=False).reset_index().head()


In [30]:
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode('New York, USA')

In [31]:
print(json.dumps(location.raw,indent=4,ensure_ascii=False))

{
    "place_id": 234930245,
    "licence": "Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright",
    "osm_type": "relation",
    "osm_id": 175905,
    "boundingbox": [
        "40.477399",
        "40.9161785",
        "-74.25909",
        "-73.7001809"
    ],
    "lat": "40.7127281",
    "lon": "-74.0060152",
    "display_name": "New York, United States of America",
    "class": "boundary",
    "type": "administrative",
    "importance": 1.017576611451846,
    "icon": "https://nominatim.openstreetmap.org/images/mapicons/poi_boundary_administrative.p.20.png"
}


## TASK: Calculate the radius parameter(Done)

In [76]:
city = 'London'
point0 = [float(city_point[city_point['city']==city].reset_index().loc[0,['lat']]),float(city_point[city_point['city']==city].reset_index().loc[0,['lng']])]
point1 = [float(city_point[city_point['city']==city].reset_index().loc[1,['lat']]),float(city_point[city_point['city']==city].reset_index().loc[1,['lng']])]
point10 =  [float(city_point[city_point['city']==city].reset_index().loc[10,['lat']]),float(city_point[city_point['city']==city].reset_index().loc[10,['lng']])]
rad1 = int(vincenty(point0, point1).meters//2)
rad2 = int(vincenty(point0, point10).meters//2)  
radius = (rad1 + rad2)//2
radius

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  """
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  


1607

## TASK: GET VENUES and create feature DataFrame(Working)

In [58]:
city_point = pd.read_csv('city_points.csv')
city_point.head(3)

,Unnamed: 0,city,lat,lng,radius
0,0,"New York, USA",40.477399,-74.259090,2669
1,1,"New York, USA",40.477399,-74.196989,2669
2,2,"New York, USA",40.477399,-74.134888,2669


In [59]:
city_point = city_point.sample(n=30, axis=0)
city_point = city_point[city_point['radius']!=0]
city_point

,Unnamed: 0,city,lat,lng,radius
110,110,"Los Angeles, USA",33.734848,-118.668178,3409
59,59,"New York, USA",40.721165,-73.700181,2669
195,195,"Los Angeles, USA",34.337306,-118.383243,3409
120,120,"Los Angeles, USA",33.810155,-118.668178,3409
354,354,"Washington, USA",38.912763,-77.054349,1759
429,429,"Paris, FR",48.834816,2.469760,768
392,392,"Washington, USA",39.054986,-77.125460,1759
7,7,"New York, USA",40.477399,-73.824383,2669
99,99,"New York, USA",40.916179,-73.700181,2669
36,36,"New York, USA",40.623659,-73.886484,2669


In [60]:
city_venues = getNearbyVenues(names=city_point['city'],
                              latitudes=city_point['lat'],
                              longitudes=city_point['lng'],
                              radiuses=city_point['radius']
                             )

33.7348482,-118.66817759999998,3409
40.7211654,-73.7001809,2669
34.337306,-118.38324270000001,3409
33.8101554,-118.66817759999998,3409
38.912763299999995,-77.0543486,1759
48.8348156,2.4697602,768
39.0549855,-77.12545970000001,1759
40.477399,-73.8243829,2669
40.9161785,-73.7001809,2669
40.6236588,-73.8864839,2669
34.0360771,-118.3262557,3409
34.1113843,-118.4972166,3409
34.1113843,-118.38324270000001,3409
33.659541,-118.44022959999998,3409
38.841652200000006,-77.12545970000001,1759
40.5261523,-73.8864839,2669
38.87720770000001,-76.9832375,1759
48.8540557,2.3878807999999996,768
33.659541,-118.2692687,3409
33.885462700000005,-118.4972166,3409
39.0549855,-77.0899041,1759
33.659541,-118.66817759999998,3409
40.6236588,-73.7001809,2669
33.960769899999995,-118.38324270000001,3409
39.0194299,-77.1965708,1759
40.6724121,-73.7622819,2669


In [61]:
city_venues.head()

,City Name,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"New York, USA",40.721165,-73.700181,"Sempre Fame Gourmet Grill, BBQ & Catering",40.720529,-73.694896,BBQ Joint
1,"New York, USA",40.721165,-73.700181,Cork and Kerry,40.724665,-73.703867,Speakeasy
2,"New York, USA",40.721165,-73.700181,Pita Park,40.724371,-73.703381,Greek Restaurant
3,"New York, USA",40.721165,-73.700181,Village Pizzeria,40.724045,-73.702496,Pizza Place
4,"New York, USA",40.721165,-73.700181,Floral Park Centennial Gardens and Bird Sanctuary,40.717424,-73.698516,Park


In [62]:
print('There are {} uniques categories.'.format(len(city_venues['Venue Category'].unique())))

There are 242 uniques categories.


In [63]:
city_onehot = pd.get_dummies(city_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
#city_onehot['Neighborhood'] = city_venues['Neighborhood'] 
city_onehot['Latitude'] = city_venues['Latitude'] 
city_onehot['Longitude'] = city_venues['Longitude'] 
# move neighborhood column to the first column
fixed_columns = [city_onehot.columns[-1]] + list(city_onehot.columns[:-1])
city_onehot = city_onehot[fixed_columns]

city_onehot.head()

,Longitude,Accessories Store,Afghan Restaurant,African Restaurant,Airport Lounge,Airport Service,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,...,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Latitude
0,-73.700181,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,40.721165
1,-73.700181,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,40.721165
2,-73.700181,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,40.721165
3,-73.700181,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,40.721165
4,-73.700181,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,40.721165


In [64]:
city_grouped = city_onehot.groupby(['Latitude','Longitude']).mean().reset_index()

In [66]:
for i in city_grouped.columns:
    print(i)

Latitude
Longitude
Accessories Store
Afghan Restaurant
African Restaurant
Airport Lounge
Airport Service
American Restaurant
Arcade
Argentinian Restaurant
Art Gallery
Art Museum
Arts & Crafts Store
Asian Restaurant
Athletics & Sports
Auto Dealership
BBQ Joint
Bagel Shop
Bakery
Bank
Bar
Baseball Field
Basketball Court
Beach
Beer Bar
Beer Garden
Beer Store
Belgian Restaurant
Big Box Store
Bike Shop
Bike Trail
Bistro
Boat or Ferry
Bookstore
Boutique
Bowling Alley
Breakfast Spot
Brewery
Bubble Tea Shop
Buffet
Burger Joint
Bus Station
Bus Stop
Café
Cajun / Creole Restaurant
Candy Store
Caribbean Restaurant
Chinese Restaurant
Chocolate Shop
Church
Clothing Store
Cocktail Bar
Coffee Shop
Comedy Club
Comic Shop
Concert Hall
Convenience Store
Cosmetics Shop
Credit Union
Cupcake Shop
Cycle Studio
Dance Studio
Deli / Bodega
Department Store
Design Studio
Dessert Shop
Dim Sum Restaurant
Diner
Discount Store
Dive Bar
Dog Run
Donut Shop
Dosa Place
Drugstore
Dumpling Restaurant
Eastern European Resta

In [36]:
city_grouped.sort_values(by='Bar',ascending=False)

,Latitude,Longitude,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,...,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Yoga Studio,Zoo
20,48.882916,2.387881,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.015152,...,0.000000,0.015152,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,38.877208,-77.089904,0.000000,0.050000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.010000,0.00,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000
11,38.948319,-76.912126,0.000000,0.021739,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.021739,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
17,40.867425,-73.948585,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.020000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.030000,0.030000,0.000000,0.000000,0.000000
16,40.769919,-74.134888,0.000000,0.000000,0.00,0.000000,0.000000,0.010000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
15,40.769919,-74.196989,0.000000,0.031250,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.020833,0.000000,0.00,0.000000,0.000000,0.010417,0.000000,0.010417,0.010417,0.000000
14,40.721165,-74.259090,0.000000,0.020000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000
12,39.054986,-77.018793,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000
19,48.844436,2.251415,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.015385,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
18,48.825196,2.415174,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## TASK: Clustering(Pending)

In [84]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 10

city_grouped_clustering = city_grouped.drop(['Latitude','Longitude'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100]

array([3, 1, 0, 0, 6, 0, 5, 0, 0, 0, 0, 0, 0, 9, 9, 8, 9, 2, 4, 7],
      dtype=int32)

In [93]:
city_grouped.drop('Cluster Labels',axis=1,inplace=True)

In [94]:
# add clustering labels
city_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

city_merged = city_point

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
city_merged = city_merged.join(city_grouped.set_index(['Latitude','Longitude']), on=['lat','lng'])


city_merged.dropna(inplace=True)

In [103]:
city_merged.loc[:,['city','Cluster Labels','Coffee Shop','Café','Wine Bar']].sort_values(by='Coffee Shop',ascending=False)

,city,Cluster Labels,Coffee Shop,Café,Wine Bar
39,"New York, USA",9.0,0.054945,0.021978,0.00
354,"Washington, USA",0.0,0.040000,0.030000,0.01
165,"Los Angeles, USA",0.0,0.040000,0.030000,0.00
59,"New York, USA",9.0,0.030000,0.010000,0.00
156,"Los Angeles, USA",0.0,0.030000,0.030000,0.00
332,"Washington, USA",0.0,0.030000,0.000000,0.00
346,"Washington, USA",0.0,0.030000,0.010000,0.00
380,"Washington, USA",0.0,0.026316,0.000000,0.00
36,"New York, USA",9.0,0.025000,0.000000,0.00
392,"Washington, USA",0.0,0.020000,0.010000,0.00


In [101]:
city_merged

,Unnamed: 0,city,lat,lng,radius,Cluster Labels,Accessories Store,Afghan Restaurant,African Restaurant,Airport Lounge,...,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
59,59,"New York, USA",40.721165,-73.700181,2669,9.0,0.000000,0.00,0.00,0.00,...,0.000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000
195,195,"Los Angeles, USA",34.337306,-118.383243,3409,5.0,0.000000,0.00,0.00,0.00,...,0.000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000
354,354,"Washington, USA",38.912763,-77.054349,1759,0.0,0.000000,0.01,0.00,0.00,...,0.000,0.000000,0.000000,0.02,0.01,0.00,0.00,0.00,0.01,0.010000
429,429,"Paris, FR",48.834816,2.469760,768,4.0,0.000000,0.00,0.00,0.00,...,0.000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000
392,392,"Washington, USA",39.054986,-77.125460,1759,0.0,0.000000,0.00,0.00,0.00,...,0.000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.020000
99,99,"New York, USA",40.916179,-73.700181,2669,2.0,0.000000,0.00,0.00,0.00,...,0.000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000
36,36,"New York, USA",40.623659,-73.886484,2669,9.0,0.000000,0.00,0.00,0.00,...,0.025,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000
156,156,"Los Angeles, USA",34.036077,-118.326256,3409,0.0,0.000000,0.00,0.00,0.00,...,0.000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.01,0.00,0.000000
163,163,"Los Angeles, USA",34.111384,-118.497217,3409,6.0,0.000000,0.00,0.00,0.00,...,0.000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000
165,165,"Los Angeles, USA",34.111384,-118.383243,3409,0.0,0.000000,0.00,0.00,0.00,...,0.000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000


In [23]:
df

,city,lat,lng
0,New York,40.477399,-74.259090
1,New York,40.477399,-74.196989
2,New York,40.477399,-74.134888
3,New York,40.477399,-74.072787
4,New York,40.477399,-74.010686
...,...,...,...
195,London,51.667322,-0.109870
196,London,51.667322,-0.074314
197,London,51.667322,-0.038759
198,London,51.667322,-0.003203


In [89]:
lat = 40.6236588
lng = -74.01068599999999
radius = 2669
LIMIT = 100
    
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

In [90]:
results = requests.get(url).json()["response"]['groups'][0]['items']

In [87]:
city_points = {}
geo_point_creator('New York')

In [25]:
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode('Seoul')

In [26]:
print(json.dumps(location.raw,indent=4,ensure_ascii=False))

{
    "place_id": 279111120,
    "licence": "Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright",
    "osm_type": "relation",
    "osm_id": 2297418,
    "boundingbox": [
        "37.4285424",
        "37.7013911",
        "126.7644328",
        "127.1832695"
    ],
    "lat": "37.5666791",
    "lon": "126.9782914",
    "display_name": "서울, 대한민국",
    "class": "boundary",
    "type": "administrative",
    "importance": 0.6297939847554467,
    "icon": "https://nominatim.openstreetmap.org/images/mapicons/poi_boundary_administrative.p.20.png"
}
